# Query  `apiso:ServiceType` on CINERGI csw

In [1]:
from owslib.csw import CatalogueServiceWeb
from owslib import fes
import numpy as np

In [2]:
endpoint = 'http://cinergi.cloudapp.net/geoportal/csw'   # NODC/UAF Geoportal: granule level
csw = CatalogueServiceWeb(endpoint,timeout=60)
print csw.version

2.0.2


In [3]:
csw.get_operation_by_name('GetRecords').constraints

[Constraint: SupportedCommonQueryables - ['Subject', 'Title', 'Abstract', 'AnyText', 'Format', 'Identifier', 'Modified', 'Type', 'BoundingBox'],
 Constraint: SupportedISOQueryables - ['apiso:Subject', 'apiso:Title', 'apiso:Abstract', 'apiso:AnyText', 'apiso:Format', 'apiso:Identifier', 'apiso:Modified', 'apiso:Type', 'apiso:BoundingBox', 'apiso:CRS.Authority', 'apiso:CRS.ID', 'apiso:CRS.Version', 'apiso:RevisionDate', 'apiso:AlternateTitle', 'apiso:CreationDate', 'apiso:PublicationDate', 'apiso:OrganizationName', 'apiso:HasSecurityConstraints', 'apiso:Language', 'apiso:ResourceIdentifier', 'apiso:ParentIdentifier', 'apiso:KeywordType', 'apiso:TopicCategory', 'apiso:ResourceLanguage', 'apiso:GeographicDescriptionCode', 'apiso:Denominator', 'apiso:DistanceValue', 'apiso:DistanceUOM', 'apiso:TempExtent_begin', 'apiso:TempExtent_end', 'apiso:ServiceType', 'apiso:ServiceTypeVersion', 'apiso:Operation', 'apiso:OperatesOn', 'apiso:OperatesOnIdentifier', 'apiso:OperatesOnName', 'apiso:Coupling

## Try a simple query first using just `apiso:AnyText`

In [4]:
val = 'roms'
#val = 'modflow'
filter1 = fes.PropertyIsLike(propertyname='apiso:AnyText',literal=('*%s*' % val),
                        escapeChar='\\',wildCard='*',singleChar='?')
filter_list = [ filter1 ]

In [5]:
csw.getrecords2(constraints=filter_list,maxrecords=100,esn='full')
print len(csw.records.keys())
for rec in list(csw.records.keys()):
    print csw.records[rec].title 
    

25
USGS 1:125000-scale Quadrangle for Stromsburg, NE 1896
ROMS CNMI Regional Ocean Model Assimilation/Best Time Series
Regional Ocean Modeling System (ROMS): Main Hawaiian Islands: Regridded
ROMS Oahu South Shore Regional Ocean Model/Best Time Series
Regional Ocean Modeling System (ROMS): Oahu: Data Assimilating
ROMS Guam Regional Ocean Model/Best Time Series
Regional Ocean Modeling System (ROMS): Main Hawaiian Islands
Regional Ocean Modeling System (ROMS): Main Hawaiian Islands: Data Assimilating
Regional Ocean Modeling System (ROMS): Oahu: Data Assimilating: Regridded
ROMS CNMI Regional Ocean Model/Best Time Series
Regional Ocean Modeling System (ROMS): Guam: Regridded
Regional Ocean Modeling System (ROMS): Main Hawaiian Islands
Regional Ocean Modeling System (ROMS): Oahu
Regional Ocean Modeling System (ROMS): CNMI: Regridded
ROMS Oahu South Shore Regional Ocean Model/Forecast Model Run Collection (2D time coordinates)
Regional Ocean Modeling System (ROMS): Oahu: Regridded
Sea ducks 

## Add bounding box constraint. 

To specify lon,lat order for bbox (which we want to do so that we can use the same bbox with either geoportal server or pycsw requests), we need to request the bounding box specifying the CRS84 coordinate reference system.   The CRS84 option is available in `pycsw 1.1.10`+. The ability to specify the `crs` in the bounding box request is available in `owslib 0.8.12`+.  For more info on the bounding box problem and how it was solved, see this [pycsw issue](https://github.com/geopython/pycsw/issues/287), this [geoportal server issue](https://github.com/Esri/geoportal-server/issues/124), and this [owslib issue](https://github.com/geopython/OWSLib/issues/201)

In [6]:
#bbox = [-87.40, 34.25, -63.70, 66.70]    # [lon_min, lat_min, lon_max, lat_max]      maine
bbox = [-162., 19.0, -154., 23.0]   # Hawaii
bbox_filter = fes.BBox(bbox,crs='urn:ogc:def:crs:OGC:1.3:CRS84')
filter_list = [fes.And([filter1, bbox_filter])]
csw.getrecords2(constraints=filter_list, maxrecords=1000)

In [7]:
print(len(csw.records.keys()))
for rec in list(csw.records.keys()):
    print('title:'+csw.records[rec].title) 
    print('identifier:'+csw.records[rec].identifier)
    print('modified:'+csw.records[rec].modified)
    print(' ')

11
title:Regional Ocean Modeling System (ROMS): Main Hawaiian Islands
identifier:roms_hiig_forecast/ROMS_Hawaii_Regional_Ocean_Model_fmrc.ncd
modified:2015-12-08T21:30:26+00:00
 
title:Regional Ocean Modeling System (ROMS): Main Hawaiian Islands: Data Assimilating
identifier:roms_hiig_assimilation/ROMS_Hawaii_Regional_Ocean_Model_Assimilation_best.ncd
modified:2015-12-08T21:30:22+00:00
 
title:Regional Ocean Modeling System (ROMS): Main Hawaiian Islands
identifier:roms_hiig_forecast/ROMS_Hawaii_Regional_Ocean_Model_best.ncd
modified:2015-12-08T21:30:25+00:00
 
title:Regional Ocean Modeling System (ROMS): Main Hawaiian Islands: Regridded
identifier:roms_hiig_forecast_regridded/ROMS_Hawaii_Regional_Ocean_Model_Regridded_best.ncd
modified:2015-12-08T21:30:28+00:00
 
title:Regional Ocean Modeling System (ROMS): Oahu: Data Assimilating: Regridded
identifier:roms_hiog_assimilation_regridded/ROMS_Oahu_Regional_Ocean_Model_Assimilation_Regridded_best.ncd
modified:2015-12-08T21:30:32+00:00
 
ti

## Add ServiceType constraint. 

In [11]:
val = 'opendap'
filter2 = fes.PropertyIsLike(propertyname='apiso:ServiceType',literal=('*%s*' % val),
                        escapeChar='\\',wildCard='*',singleChar='?')
filter_list = [fes.And([filter1, filter2, bbox_filter])]
csw.getrecords2(constraints=filter_list, maxrecords=1000, esn='full')

In [12]:
print(len(csw.records.keys()))
for rec in list(csw.records.keys()):
    print('title:'+csw.records[rec].title) 
    print('identifier:'+csw.records[rec].identifier)
    print('modified:'+csw.records[rec].modified)
    print(' ')

10
title:Regional Ocean Modeling System (ROMS): Main Hawaiian Islands
identifier:roms_hiig_forecast/ROMS_Hawaii_Regional_Ocean_Model_fmrc.ncd
modified:2015-12-08T21:30:26+00:00
 
title:Regional Ocean Modeling System (ROMS): Main Hawaiian Islands: Data Assimilating
identifier:roms_hiig_assimilation/ROMS_Hawaii_Regional_Ocean_Model_Assimilation_best.ncd
modified:2015-12-08T21:30:22+00:00
 
title:Regional Ocean Modeling System (ROMS): Main Hawaiian Islands
identifier:roms_hiig_forecast/ROMS_Hawaii_Regional_Ocean_Model_best.ncd
modified:2015-12-08T21:30:25+00:00
 
title:Regional Ocean Modeling System (ROMS): Main Hawaiian Islands: Regridded
identifier:roms_hiig_forecast_regridded/ROMS_Hawaii_Regional_Ocean_Model_Regridded_best.ncd
modified:2015-12-08T21:30:28+00:00
 
title:Regional Ocean Modeling System (ROMS): Oahu: Data Assimilating: Regridded
identifier:roms_hiog_assimilation_regridded/ROMS_Oahu_Regional_Ocean_Model_Assimilation_Regridded_best.ncd
modified:2015-12-08T21:30:32+00:00
 
ti

## Print out the references.   Expecting to see the opendap link here:

In [13]:
choice=np.random.choice(list(csw.records.keys()))
print(csw.records[choice].title)
csw.records[choice].references

Regional Ocean Modeling System (ROMS): Main Hawaiian Islands: Data Assimilating


[{'scheme': 'urn:x-esri:specification:ServiceType:ArcIMS:Metadata:Server',
  'url': 'http://oos.soest.hawaii.edu/thredds/wms/hioos/roms_assim/hiig/ROMS_Hawaii_Regional_Ocean_Model_Assimilation_best.ncd?service=WMS&version=1.3.0&request=GetCapabilities'},
 {'scheme': 'urn:x-esri:specification:ServiceType:ArcIMS:Metadata:Onlink',
  'url': 'http://oos.soest.hawaii.edu/thredds/dodsC/hioos/roms_assim/hiig/ROMS_Hawaii_Regional_Ocean_Model_Assimilation_best.ncd.html'},
 {'scheme': 'urn:x-esri:specification:ServiceType:ArcIMS:Metadata:Onlink',
  'url': 'http://www.ncdc.noaa.gov/oa/wct/wct-jnlp-beta.php?singlefile=http://oos.soest.hawaii.edu/thredds/dodsC/hioos/roms_assim/hiig/ROMS_Hawaii_Regional_Ocean_Model_Assimilation_best.ncd'},
 {'scheme': 'urn:x-esri:specification:ServiceType:ArcIMS:Metadata:Document',
  'url': 'http://localhost:8080/geoportal/csw?getxml=%7B57A04430-E67A-41F7-92A9-B16380E5C21D%7D'}]